In [1]:
from dateutil.parser import parse
from IPython.display import display
import numpy as np
import pandas as pd
import ast
import datetime
import nltk
import re
import string

In [2]:
reviews = pd.read_csv('../scraping/data/reviews.csv')
details = pd.read_csv('../scraping/data/details.csv')

# Etapa 2 — Pré-processamento de dados
## 2.1 Tratamento de dados ausentes

Inicialmente, visto que realizamos a coleta de um vasto conjunto de dados durante uma margem considerável de tempo, o primeiro passo para nosso pré-processamento foi realizar a verificação de dados nulos.

In [3]:
display(details.isnull().sum())

movie            1
release_date     1
acclaim_rate     1
user_score       1
director        20
genres          37
languages        0
dtype: int64

É possível verificar que, algumas colunas, a exemplo do diretor e do gênero, corresponderam ao esperado de possuírem dados ausentes, visto que havia alguns filmes que não contavam com estas infomações em sua página de detalhes. Por outro lado, a relação acima permite-nos concluir que há um filme sem o atributo "movie" — que corresponde ao nome do mesmo. Visto que se trata de um dado trivial nesse sentido, partimos para a verificação do ocorrido com mais detalhes.

In [4]:
display(details[details.movie.isnull()])

,movie,release_date,acclaim_rate,user_score,director,genres,languages
5208,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [5]:
details = details.drop(5208); reviews = reviews.drop(5208)
details.reset_index(inplace=True, drop=True)
reviews.reset_index(inplace=True, drop=True)

É possível ver que o mesmo se tratava, na verdade, de um dado corrompido — perdido por alguma motivação durante a realização da coleta. Como todos os seus valores são nulos, é possível ver que o mesmo não trás contribuição nenhuma para nosso conjunto de dados, então eliminamos de ambos os DataFrames. Quanto ao tratamento de dados ausentes, a solução que optamos fora ignorar estes dados, visto que a realização do mesmo não irá prejudicar muito nossas análises posteriormente. Realizamos apenas a substituição do diretor para facilitar sua categorização a seguir.

In [6]:
details.director = details.director.fillna('unknown')

## 2.2 Ajuste de tipos de atributos

Visto que o conjunto de dados coletados consiste de dados de diversos, fora necessário conduzir conversões de tipos que melhor representem os dados com quais estamos trabalhando. Iniciaremos pelo DataFrame de reviews.

In [7]:
print(reviews.dtypes)

movie           object
reviewer        object
metascore       object
review_score     int64
review_text     object
review_date     object
details         object
dtype: object


É possível ver que a coluna das avaliações das resenhas foi representada corretamente, mas as demais não foram. O Metascore, por exemplo, também se trata de um critério de avaliação, então não faz sentido representá-lo como um objeto. Isso ocorre devido à presença de valores ausentes nos dados. O Metascore, por exemplo, possui valores que ainda não foram determinados — seja por que o filme ainda não foi lançado ou por que o número de reviews global é insuficiente para determiná-lo. Esses dados, então, ficam definidos como *tbd* (do inglês, *to be defined*), convertendo toda a coluna. O mesmo ocorre para filmes onde a data de lançamento ainda não foi anunciada. Assim, fora necessário criar duas funções capazes de realizar estas conversões.

In [8]:
def metascore_parser(score):
    if score != 'tbd':
        return int(score)
    return np.nan

def date_parser(date):
    if date is np.nan or date == 'TBA':
        return np.nan
    return parse(date)

Isso feito, o próximo passo for aplicá-las às colunas do Metascore e das datas das resenhas. Por sua vez, como estaremos conduzindo análises sobre a influência dos mesmos mais adiante, os críticos seriam melhor representados como categorias. Por fim, a coluna de detalhes não é mais necessária, visto que já realizamos a coleta dos detalhes dos filmes.

In [9]:
reviews.metascore = reviews['metascore'].apply(metascore_parser)
reviews.review_date = reviews['review_date'].apply(date_parser)
reviews.reviewer = reviews['reviewer'].astype('category')
reviews = reviews.drop('details', axis=1)

display(reviews.head())
display(reviews.dtypes)

,movie,reviewer,metascore,review_score,review_text,review_date
0,The Grandmaster,Manohla Dargis,73.0,100,"The Grandmaster is, at its most persuasive, ab...",2013-08-22
1,Of Gods and Men,A.O. Scott,86.0,100,"Of Gods and Men is supple and suspenseful, app...",2011-02-24
2,Kubo and the Two Strings,Glenn Kenny,84.0,100,"The action is gorgeously fluid, the idiosyncra...",2016-08-18
3,Jane,Ben Kenigsberg,87.0,100,Jane will delight those familiar with Ms. Good...,2017-10-19
4,Bird People,A.O. Scott,70.0,100,"There are plot twists, and then there is what ...",2014-09-11


movie                   object
reviewer              category
metascore              float64
review_score             int64
review_text             object
review_date     datetime64[ns]
dtype: object

Em seguida, partimos para as mesmas transformações, agora para o DataFrame de detalhes.

In [10]:
print(details.dtypes)

movie           object
release_date    object
acclaim_rate    object
user_score      object
director        object
genres          object
languages       object
dtype: object


Podemos observar que a conversão dos tipos também não foi corretamente empregada. A data de lançamento do filme, mesmo que não tenha sido anunciada como visto anteriormente, poderia ser melhor representada em uma série temporal — algo que já definimos em funções acima. Similarmente, o diretor, gênero e os idiomas dos filmes seriam melhor representados como categóricos, visto que irão compor parte de nossa análise posteriormente. Por fim, a aclamação dos usuários também seria melhor representada desta forma, enquanto o escore global dos usuários deveria estar representado como um número, e não como um objeto. Assim sendo, seguimos com as conversões dos tipos.

In [11]:
def userscore_parser(score):
    if score != 'tbd':
        return float(score)*10
    return np.nan

list_parser = lambda x: ast.literal_eval(x) if x is not np.nan else ['unknown']
getFirstElem = lambda y: y[0] if len(y) is not 0 else 'unknown'

In [12]:
details.genres = details['genres'].apply(list_parser)
binary_genres = pd.get_dummies(details['genres'].apply(pd.Series).stack()).sum(level=0)

details = details.drop('genres', axis=1)
details = pd.concat([details, binary_genres], axis=1)

A conversão dos gêneros em variáveis *dummy* nos auxiliará ao decorrer de nossas análises, visto que esta é a melhor maneira de representá-los como atributos categóricos de diversos níveis. Além disso, pudemos verificar que o idioma, em grande parte, não é composto apenas por valores significativos. O filme The Grandmaster, por exemplo, possui em seu campo de idiomas: mandarim, japonês e cantonês. Apesar disso, o idioma principal do filme é apenas o mandarim. O mesmo ocorre para tipos diferentes de filmes. Assim sendo, realizamos a coleta apenas do primeiro idioma, visto que este é suficientemente representativo do filme. Em seguida, realizamos as conversões.

In [13]:
details.user_score = details['user_score'].apply(userscore_parser)
details.release_date = details['release_date'].apply(date_parser)
details.languages = details['languages'].apply(list_parser).apply(getFirstElem)
details.languages = details['languages'].astype('category')
details.acclaim_rate = details['acclaim_rate'].astype('category')
details.director = details['director'].astype('category')
for col in details.columns[6:-1]:
    details[col] = details[col].astype('int64')

display(details.head())
display(details.dtypes)

,movie,release_date,acclaim_rate,user_score,director,languages,Action,Adult,Adventure,Animation,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,unknown
0,The Grandmaster,2013-08-23,Generally favorable reviews,67.0,Kar Wai Wong,Mandarin,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Of Gods and Men,2011-02-25,Generally favorable reviews,70.0,Xavier Beauvois,French,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Kubo and the Two Strings,2016-08-19,Universal acclaim,81.0,Travis Knight,English,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Jane,2017-10-20,No score yet,NaN,Brett Morgen,English,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bird People,2014-09-12,Generally favorable reviews,70.0,Pascale Ferran,English,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


movie                   object
release_date    datetime64[ns]
acclaim_rate          category
user_score             float64
director              category
languages             category
Action                   int64
Adult                    int64
Adventure                int64
Animation                int64
Biography                int64
Comedy                   int64
Crime                    int64
Documentary              int64
Drama                    int64
Family                   int64
Fantasy                  int64
Film-Noir                int64
History                  int64
Horror                   int64
Music                    int64
Musical                  int64
Mystery                  int64
News                     int64
Reality-TV               int64
Romance                  int64
Sci-Fi                   int64
Short                    int64
Sport                    int64
Talk-Show                int64
Thriller                 int64
War                      int64
Western 

## 2.3 Discretização

Também fora necessário empregar técnicas de discretização para direcionarmos melhor nosso estudo. As décadas de lançamento, por exemplo, também são potenciais fatores que podemos empregar em nossas análises.

In [14]:
details['release_decade'] = pd.cut(details.release_date.map(lambda x: x.year),
       bins=np.insert(np.linspace(1930, 2010, 9, dtype=np.int32), 9, 2017),
       labels=list(map(lambda x: '{}s'.format(x),
       np.linspace(1930, 2010, 9, dtype=np.int32)))
)

display(pd.concat([details.release_date.apply(lambda x: x.year).head(10),
                   details.release_decade.head(10)], axis=1))

,release_date,release_decade
0,2013.0,2010s
1,2011.0,2010s
2,2016.0,2010s
3,2017.0,2010s
4,2014.0,2010s
5,2012.0,2010s
6,2015.0,2010s
7,1999.0,1990s
8,2011.0,2010s
9,2012.0,2010s


Também seria útil realizar a discretização para o escore dos críticos, visto que o sentimento pode ser tornar algo necessário quando empregarmos a classificação de texto.

In [20]:
def get_sentiment(score):
    if score is np.nan:
        return np.nan
    if score < 40:
        return 'neg'
    elif score > 60:
        return 'pos'
    else:
        return 'mixed'
    
reviews['sentiment'] = reviews['review_score'].apply(get_sentiment)
reviews['sentiment'] = reviews['sentiment'].astype('category')

## 2.4 Integração de dados

Para concluirmos, o único fator que ainda restava era a realização da integração dos dados. Ao tentar realizar o mesmo através da função *merge* de Pandas, vimos que ocorria um aumento do número de linhas. Conduzindo análises, pudemos encontrar duplicação em nossos dados, o que inicialmente levantou nossas suspeitas de que poderiam se tratar de resenhas diferentes. Ao olhar o DataFrame de detalhes, podemos ver que se tratam exatamente do mesmo filme.

In [16]:
display(details[details.duplicated()])
display(details[details.movie=='I Love You, Daddy'])

,movie,release_date,acclaim_rate,user_score,director,languages,Action,Adult,Adventure,Animation,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,unknown,release_decade
10525,"I Love You, Daddy",NaT,No score yet,NaN,Louis C.K.,English,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


,movie,release_date,acclaim_rate,user_score,director,languages,Action,Adult,Adventure,Animation,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,unknown,release_decade
2779,"I Love You, Daddy",NaT,No score yet,NaN,Louis C.K.,English,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
10525,"I Love You, Daddy",NaT,No score yet,NaN,Louis C.K.,English,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


Porém, uma análise mais cuidadosa no DataFrame de reviews nos permite concluir que são realmente resenhas distintas. Além de o escore empregado em cada uma das avaliações ser diferente, o texto da review nos mostra claramente que a crítica assistiu o filme duas vezes e teve conclusões distintas.

In [21]:
display(reviews[reviews.movie=='I Love You, Daddy'])

,movie,reviewer,metascore,review_score,review_text,review_date,sentiment
2779,"I Love You, Daddy",Manohla Dargis,56.0,80,"“Zama” and I Love You, Daddy are two of the be...",NaT,pos
10525,"I Love You, Daddy",Manohla Dargis,56.0,40,"When I watched I Love You, Daddy a second time...",NaT,mixed


Por se tratarem de resenhas diferentes, não faz sentido considerarmos os mesmos como dados duplicados, ainda que se tratem do mesmo filme. Assim sendo, adicionamos parâmetros adicionais para realizarmos a integração dos dados pelo índice nestas ocorrências.

In [22]:
movie_reviews = pd.merge(reviews, details, on='movie', left_index=True, right_index=True)
display(movie_reviews.head())
display(movie_reviews.shape)

,movie,reviewer,metascore,review_score,review_text,review_date,sentiment,release_date,acclaim_rate,user_score,...,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,unknown,release_decade
0,The Grandmaster,Manohla Dargis,73.0,100,"The Grandmaster is, at its most persuasive, ab...",2013-08-22,pos,2013-08-23,Generally favorable reviews,67.0,...,0,0,0,0,0,0,0,0,0,2010s
1,Of Gods and Men,A.O. Scott,86.0,100,"Of Gods and Men is supple and suspenseful, app...",2011-02-24,pos,2011-02-25,Generally favorable reviews,70.0,...,0,0,0,0,0,0,0,0,0,2010s
2,Kubo and the Two Strings,Glenn Kenny,84.0,100,"The action is gorgeously fluid, the idiosyncra...",2016-08-18,pos,2016-08-19,Universal acclaim,81.0,...,0,0,0,0,0,0,0,0,0,2010s
3,Jane,Ben Kenigsberg,87.0,100,Jane will delight those familiar with Ms. Good...,2017-10-19,pos,2017-10-20,No score yet,NaN,...,0,0,0,0,0,0,0,0,0,2010s
4,Bird People,A.O. Scott,70.0,100,"There are plot twists, and then there is what ...",2014-09-11,pos,2014-09-12,Generally favorable reviews,70.0,...,1,0,0,0,0,0,0,0,0,2010s


(12410, 41)

Por fim, visto que coletamos os dados diretamente da página da publicação no site — e que esta os ordenava conforme o escore de avaliação do crítico do The New York Times — aleatorizamos os dados para evitar problemas posteriores. Realizamos, ainda, a renomação de algumas colunas para torná-las mais mnemônicas. Adiante, salvamos tudo em um único arquivo *csv* para concluirmos o pré-processamento e partirmos para a análise.

In [24]:
movie_reviews = movie_reviews.sample(frac=1).reset_index(drop=True)
movie_reviews.rename(columns={'Sci-Fi': 'SciFi', 'Reality-TV': 'RealityTV', 
                           'Talk-Show': 'TalkShow', 'Film-Noir': 'FilmNoir',
                           'acclaim_rate': 'user_acclaim'}, 
                  inplace=True)
movie_reviews.to_csv('../movie_data.csv', sep=',', index=False)